In [1]:
import pygame
import random
import os

###################################################################################
#초기화
pygame.init()

#화면 크기 설정
screen_width = 640
screen_height = 480
screen = pygame.display.set_mode((screen_width, screen_height))

#화면 타이틀 설정
pygame.display.set_caption("YW Pang")

#FPS
clock = pygame.time.Clock()
###################################################################################

# 1. 기초 게임 설정

current_path = os.path.dirname(os.path.realpath('__file__')) #현재 파일 위치 반환
image_path = os.path.join(current_path, "images") # images 폴더 위치 반환

#배경 이미지
background = pygame.image.load(os.path.join(image_path, "background.png"))

#스테이지 이미지
stage = pygame.image.load(os.path.join(image_path, "stage.png"))
stage_size = stage.get_rect().size
stage_height = stage_size[1] #스테이지 위에 캐릭터 놓기 위해 높이 설정

#캐릭터
character = pygame.image.load(os.path.join(image_path, "character.png"))
character_size = character.get_rect().size #이미지 크기
character_width = character_size[0] #캐릭터 가로
character_height = character_size[1] #캐릭터 세로
character_x_pos = (screen_width - character_width) / 2 #캐릭터 x 위치
character_y_pos = screen_height - character_height - stage_height #캐릭터 y 위치

#이동
to_x = 0

#이동 속도
charater_speed = 5
weapon_speed = 10
ball_speed_y = [-18, -15, -12, -9]

#무기
weapon = pygame.image.load(os.path.join(image_path, "weapon.png"))
weapon_size = weapon.get_rect().size #이미지 크기
weapon_width = weapon_size[0] #무기 가로

#무기는 한 번에 여러 발 발사 가능
weapons = []

#공
ball_images = [
    pygame.image.load(os.path.join(image_path, "balloon1.png")),
    pygame.image.load(os.path.join(image_path, "balloon2.png")),
    pygame.image.load(os.path.join(image_path, "balloon3.png")),
    pygame.image.load(os.path.join(image_path, "balloon4.png"))]
balls = []

#최초 발생 큰 공
balls.append({
    "pos_x" : 50, #공 x좌표
    "pos_y" : 50, #공 y좌표
    "img_idx" : 0, #공 인덱스
    "to_x" : -3, #x축 이동방향
    "to_y" : -6, #y축 이동방향
    "init_spd_y" : ball_speed_y[0] #y축 속도
})

#사라질 무기와 공 저장 변수
weapon_to_remove = -1
ball_to_remove = -1

#Font 정의
game_font = pygame.font.Font(None, 40)

#시간 설정
total_time = 100
start_ticks = pygame.time.get_ticks()

#게임 엔딩
game_result = "Game Over"


###################################################################################

#이벤트 루프
running = True #게임 실행 중
while running:
    dt = clock.tick(30) #게임 FPS
    #이벤트 처리
    for event in pygame.event.get(): #이벤트 확인
        if event.type == pygame.QUIT:#창 닫는 이벤트
            running = False #게임 종료
        if event.type == pygame.KEYDOWN: #키 다운 확인
            if event.key == pygame.K_LEFT: #키 왼쪽
                to_x -= charater_speed
            elif event.key == pygame.K_RIGHT: #키 오른쪽
                to_x += charater_speed
            elif event.key == pygame.K_SPACE: #무기 발사
                weapon_x_pos = character_x_pos + (character_width - weapon_width)/2
                weapon_y_pos = character_y_pos
                weapons.append([weapon_x_pos, weapon_y_pos])
                
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                to_x = 0

    #캐릭터 이동
    character_x_pos += to_x
    
    #가로 경계 생성
    if character_x_pos < 0:
        character_x_pos = 0
    elif character_x_pos > screen_width - character_width:
        character_x_pos = screen_width - character_width
        
    #무기 위치
    weapons = [ [w[0], w[1] - weapon_speed] for w in weapons]
    
    #무기 사라지기
    weapons = [ [w[0], w[1]] for w in weapons if w[1] > 0]
        
    #공 위치 정의
    for ball_idx, ball_val in enumerate(balls):
        ball_pos_x = ball_val["pos_x"]
        ball_pos_y = ball_val["pos_y"]
        ball_img_idx = ball_val["img_idx"]
        
        ball_size = ball_images[ball_img_idx].get_rect().size
        ball_width = ball_size[0]
        ball_height = ball_size[1]
        
        #x경계 튕김
        if ball_pos_x < 0 or ball_pos_x > screen_width - ball_width:
            ball_val["to_x"] = ball_val["to_x"] * -1
        #스테이지 튕김
        if ball_pos_y >= screen_height - stage_height - ball_height:
            ball_val["to_y"] = ball_val["init_spd_y"]
        else:
            ball_val["to_y"] += 0.5
        
        ball_val["pos_x"] += ball_val["to_x"]
        ball_val["pos_y"] += ball_val["to_y"]
        
    #충돌 이벤트
    character_rect = character.get_rect() #캐릭터 위치 정보 업데이트
    character_rect.left = character_x_pos
    character_rect.top = character_y_pos
    
    for ball_idx, ball_val in enumerate(balls):
        ball_pos_x = ball_val["pos_x"]
        ball_pos_y = ball_val["pos_y"]
        ball_img_idx = ball_val["img_idx"]
        
        ball_rect = ball_images[ball_img_idx].get_rect() #공 rect
        ball_rect.left = ball_pos_x
        ball_rect.top = ball_pos_y
        
        if character_rect.colliderect(ball_rect): #공과 캐릭터 충돌 처리
            running = False
            break
        
        for weapon_idx, weapon_val in enumerate(weapons):
            weapon_pos_x = weapon_val[0]
            weapon_pos_y = weapon_val[1]
            
            weapon_rect = weapon.get_rect() #무기 rect
            weapon_rect.left = weapon_pos_x
            weapon_rect.top = weapon_pos_y
            
            if weapon_rect.colliderect(ball_rect): #공과 무기 충돌 처리
                weapon_to_remove = weapon_idx
                ball_to_remove = ball_idx
                
                #공 나누기
                if ball_img_idx < 3:
                    #현재 공 정보
                    ball_width = ball_rect.size[0]
                    ball_height = ball_rect.size[1]
                    
                    #나뉜 공 정보
                    small_ball_rect = ball_images[ball_img_idx + 1].get_rect()
                    small_ball_width = small_ball_rect.size[0]
                    small_ball_height = small_ball_rect.size[1]
                    
                    #왼쪽
                    balls.append({
                    "pos_x" : ball_pos_x + (ball_width - small_ball_width)/2, #공 x좌표
                    "pos_y" : ball_pos_y + (ball_height - small_ball_height)/2, #공 y좌표
                    "img_idx" : ball_img_idx + 1, #공 인덱스
                    "to_x" : -3, #x축 이동방향
                    "to_y" : -6, #y축 이동방향
                    "init_spd_y" : ball_speed_y[ball_img_idx + 1] #y축 속도
                    })
                    
                    #오른쪽
                    balls.append({
                    "pos_x" : ball_pos_x + (ball_width - small_ball_width)/2, #공 x좌표
                    "pos_y" : ball_pos_y + (ball_height - small_ball_height)/2, #공 y좌표
                    "img_idx" : ball_img_idx + 1, #공 인덱스
                    "to_x" : 3, #x축 이동방향
                    "to_y" : -6, #y축 이동방향
                    "init_spd_y" : ball_speed_y[ball_img_idx + 1] #y축 속도
                    })  
                break
        else:
            continue
        break
                
    
    #충돌 공과 무기 제거
    if ball_to_remove > -1:
        del balls[ball_to_remove]
        ball_to_remove = -1
        
    if weapon_to_remove > -1:
        del weapons[ball_to_remove]
        weapon_to_remove = -1
    
    if len(balls) == 0:
        game_result = "Mission Complete"
        running = False
    
    screen.blit(background, (0, 0)) #배경 그리기
    for weapon_x_pos, weapon_y_pos in weapons:# 무기 그리기
        screen.blit(weapon, (weapon_x_pos, weapon_y_pos))
    for idx, val in enumerate(balls):#공 그리기
        ball_pos_x = val["pos_x"]
        ball_pos_y = val["pos_y"]
        ball_img_idx = val["img_idx"]
        screen.blit(ball_images[ball_img_idx], (ball_pos_x, ball_pos_y))
    screen.blit(stage, (0, screen_height - stage_height)) #스테이지 그리기
    screen.blit(character, (character_x_pos, character_y_pos)) #캐릭터 그리기

    #경과 시간 계산
    elapsed_time = (pygame.time.get_ticks() - start_ticks) / 1000
    timer = game_font.render("Time : {}".format(int(total_time - elapsed_time)), True, (255, 255, 255))
    screen.blit(timer, (10, 10))
    
    #시간초과 엔딩
    if total_time - elapsed_time <= 0:
        game_result = "Time Over"
        running = False
    
    pygame.display.update() #화면 업데이트


#게임오버 메시지
msg = game_font.render(game_result, True,(255, 255, 0))
msg_rect = msg.get_rect(center = (int(screen_width / 2), int((screen_height - stage_height) / 2)))
screen.blit(msg, msg_rect)

pygame.display.update()
pygame.time.delay(2000)

#게임 종료
pygame.quit()

pygame 2.0.1 (SDL 2.0.14, Python 3.7.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
